In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Get State Data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [4]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get COVID Data

In [12]:
def get_deaths(row: pd.Series):
    state_code = row["STUSPS"]
    url = f"https://jhucoronavirus.azureedge.net/api/v2/regions/us/{state_code.lower()}.json"
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()["confirmed_cases"]["all"]
    else:
        print(row["NAME"])

In [13]:
states_df["cases"] = states_df.apply(lambda row: get_deaths(row), axis=1)

## Get Population Data

In [14]:
state_populations = pd.read_excel(
    "data/NST-EST2023-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [15]:
state_populations_df = state_populations["NST-EST2023-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

## Merge Data

In [16]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "POPULATION", "geometry", "NAME", "cases"]
]

In [17]:
states_with_population_df = states_with_population_df.dropna()

In [23]:
states_with_population_df["per_1000"] = states_with_population_df["cases"] / (
    states_with_population_df["POPULATION"] / 1000
)
states_with_population_df["per_10k"] = states_with_population_df["cases"] / (
    states_with_population_df["POPULATION"] / 10_000
)
states_with_population_df["per_100k"] = states_with_population_df["cases"] / (
    states_with_population_df["POPULATION"] / 100000
)
states_with_population_df["per_500k"] = states_with_population_df["cases"] / (
    states_with_population_df["POPULATION"] / 500_000
)
states_with_population_df["per_1m"] = states_with_population_df["cases"] / (
    states_with_population_df["POPULATION"] / 1_000_000
)
states_with_population_df["per_capita"] = (
    states_with_population_df["cases"] / states_with_population_df["POPULATION"]
)

In [24]:
states_with_population_df = states_with_population_df.to_crs(9311)

In [25]:
states_with_population_df.to_file("data/covid_cases.gpkg")